In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [19]:
data=pd.read_csv("HIGGS.csv/HIGGS.csv",nrows=50000,header=None)

In [20]:
test=pd.read_csv("HIGGS.csv/HIGGS.csv",nrows=50000,skiprows=10500000,header=None)

In [21]:
rf_Kfold=KFold(n_splits=5)
lr_Kfold=KFold(n_splits=4)
rf=RandomForestClassifier()
train_data=data.iloc[:,1:]
target_data=data.iloc[:,0]

In [22]:
rf_oof_predict=np.zeros(data.shape[0])
for train_idx, test_idx in rf_Kfold.split(data):
    rf.fit(train_data.loc[train_idx],target_data.loc[train_idx])
    rf_oof_predict[test_idx]=rf.predict_proba(train_data.loc[test_idx])[:,1]
    print("Fold ROC score is %f" % roc_auc_score(target_data[test_idx],rf_oof_predict[test_idx]))
    

Fold ROC score is 0.734856
Fold ROC score is 0.733782
Fold ROC score is 0.733572
Fold ROC score is 0.729510
Fold ROC score is 0.725742


In [23]:
lr=LogisticRegression()
lr_oof_predict=np.zeros(data.shape[0])
for train_idx, test_idx in lr_Kfold.split(data):
    lr.fit(train_data.iloc[train_idx],target_data.iloc[train_idx])
    lr_oof_predict[test_idx]=lr.predict_proba(train_data.loc[test_idx])[:,1]
    print("Fold ROC score is %f" % roc_auc_score(target_data[test_idx],lr_oof_predict[test_idx]))
    
    
    

Fold ROC score is 0.687455
Fold ROC score is 0.681206
Fold ROC score is 0.675599
Fold ROC score is 0.676999


In [9]:
rf_oof_predict

array([0.9, 0.8, 0.7, ..., 0.1, 0.5, 0.8])

In [24]:
from sklearn.linear_model import LinearRegression
stage2=LinearRegression()


In [25]:
stage2_data=pd.DataFrame({'RF_data':rf_oof_predict,'Log_Data':lr_oof_predict})

In [26]:
stage2_data

,RF_data,Log_Data
0,0.8,0.623785
1,0.8,0.621718
2,0.9,0.681501
3,0.8,0.599843
4,0.7,0.492166
5,0.1,0.436956
6,0.8,0.659486
7,0.7,0.645068
8,0.5,0.898230
9,0.9,0.695113


In [27]:
stage2.fit(stage2_data,target_data)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [28]:
stage2.coef_

array([0.67966431, 0.52413764])

In [29]:
lr.fit(train_data,target_data)
rf.fit(train_data,target_data)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [30]:
test_lr=lr.predict_proba(test.loc[:,1:])[:,1]
print("Log Reg score is %f" % roc_auc_score(test.loc[:,0],test_lr))
test_rf=rf.predict_proba(test.loc[:,1:])[:,1]
print("RF score is %f" % roc_auc_score(test.loc[:,0],test_rf))
test_s2=pd.DataFrame({'RF data':test_rf,'Log Data':test_lr})
test_stage2=stage2.predict(test_s2)
print("Final Stage score is % f" % roc_auc_score(test.loc[:,0],test_stage2))

Log Reg score is 0.683865
RF score is 0.742051
Final Stage score is  0.756542


In [ ]:
data.shape

In [ ]:
rf.fit(train_data,target_data)
lr.fit(train_data,target_data)

In [17]:
rf_preds=rf.predict_proba(train_data)[:,1]
lr_preds=lr.predict_proba(train_data)[:,1]
train_data_s2=pd.DataFrame({'RF data':rf_preds,'Log Data':lr_preds})
stage2.fit(train_data_s2,target_data)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
rf_test=rf.predict_proba(test.loc[:,1:])[:,1]
print("RF score is %f" % roc_auc_score(test.loc[:,0],rf_test))
lr_test=lr.predict_proba(test.loc[:,1:])[:,1]
print("Log Reg score is %f" % roc_auc_score(test.loc[:,0],lr_test))
test_data_s2=pd.DataFrame({'RF data':rf_test,'Log Data':lr_test})
final=stage2.predict(test_data_s2)
print("Final score is %f" % roc_auc_score(test.loc[:,0],final))

RF score is 0.717866
Log Reg score is 0.677535
Final score is 0.706069


In [ ]:
test_s2['target']=target_data

In [ ]:
test_s2